# 심층 신경망 

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255
train_scaled = train_scaled.reshape(-1, 28 * 28)

train_scaled, val_scaled, train_target, val_target = train_test_split( \
    train_scaled, train_target, test_size = 0.2, random_state=42)

## 은닉층 ( 층 추가 )
입력층과 출력층 사이에 밀집층이 추가되었다  
출력층에 적용하는 활성화 함수는 종류가 제한되어있다 ( 이진분류 : 시그모이드, 다중 분류 : 소프트맥스 ).  
은닉층 활성화 함수는 비교적 자유롭다. 대표적으로 시그모이드 함수와 렐루 함수 등을 사용한다.  
선형 계산을 적당하게 비선형적으로 비털어 주저야 다음 층의 계산과 단순히 합쳐지지 않는다.

---
회귀 출력은 활성화 함수를 적용ㅇ하지 않는다. 출력층의 선형 방정식의 계산 그대로를 출력한다! <br>
  이를 위해서는 `Dense(activation = None)` 으로 지정한다.  

#### 모델 생성 방법 1

In [3]:
# 100개의 뉴런을 가진 밀집층이며, 활성화 함수 시그모이드, 입력 크기 (784, )
# 은닉층 뉴런 개수는 특별한 기준이 없다 > 개인 역량 
# 그럼에도 불구하고...? 적어도 출력층의 뉴런보다는 많게 만들어야 한다.
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784, ), name = 'first')
dense2 = keras.layers.Dense(10, activation = 'softmax', name = 'second')

In [4]:
# 리스트틑 가장 처음 등장하는 은닉층에서 마지막 출력층의 순서로 나열해야 한다.
model = keras.Sequential([dense1, dense2])

### model.summary()


Model: 모델의 이름

---

층 이름 (클래스)                  출력 크기                  모델 파라미터 개수
Layer (type)                 Output Shape              Param #   

=================================================================

first (Dense)                (None, 100)                78500 = 784개 * 100개 + 100개     
    (100 > 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축됨)
_________________________________________________________________

second (Dense)               (None, 10)                1010 = 100개 * 10개 + 10개      
 100개의 은닉층 뉴런과 1-개의 출력층 뉴런이 모두 연결되고, 출력층의 뉴련마다 하나의 절편이 있음
 
=================================================================

Total params: 79,510 은닉층과 출력층의 파라미터 개수를 합친 값
Trainable params: 79,510
Non-trainable params: 0 훈련되지 않은 파라미터 
_________________________________________________________________

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first (Dense)                (None, 100)               78500     
_________________________________________________________________
second (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


#### 모델 생성 방법 2

In [6]:
model = keras.Sequential([\
                         keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784, ), name = 'hidden'), 
                         keras.layers.Dense(10, activation = 'softmax', name = 'output'), ],
                        name = '패션 MNIST 모델')

In [7]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


#### 모델 생성 방법 3

In [8]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784, ), name = 'hidden'))
model.add(keras.layers.Dense(10, activation = 'softmax', name = 'output'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 훈련

In [10]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs= 5)

Epoch 1/5
1500/1500 [==============================] - 1s 643us/step - loss: 0.7616 - accuracy: 0.7541
Epoch 2/5
1500/1500 [==============================] - 1s 637us/step - loss: 0.4149 - accuracy: 0.8509
Epoch 3/5
1500/1500 [==============================] - 1s 622us/step - loss: 0.3740 - accuracy: 0.8645
Epoch 4/5
1500/1500 [==============================] - 1s 607us/step - loss: 0.3529 - accuracy: 0.8718
Epoch 5/5
1500/1500 [==============================] - 1s 603us/step - loss: 0.3344 - accuracy: 0.8775


In [11]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 371us/step - loss: 0.3473 - accuracy: 0.8744


[0.3473353683948517, 0.8744166493415833]

# ReLU 함수
이미지 분류 문제에서 높은 성능을 낼 수 있다.   
입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 입력을 통과시키고, 음수일 경우는 0으로 변환함.    
`max(0, z)` : z가 0보다 크면 z를 출력하고, 0보다 작으면 0을 출력한다.
<br><br>
## Flatten 층
numpy의 reshape을 대신해 사용할 수 있다.  
flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할을 한다.   
입력에 곱해지는 가중치나 절편이 없다.
성능에 기여하지는 않지만, 입력층과 은닉층 사이에 추가하기 때문에 층이라 부른다!!

In [12]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---

In [14]:
(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255
# train_scaled = train_scaled.reshape(-1, 28 * 28)

train_scaled, val_scaled, train_target, val_target = train_test_split( \
    train_scaled, train_target, test_size = 0.2, random_state=42)

In [15]:
model.compile(loss='sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 1s 609us/step - loss: 0.6808 - accuracy: 0.7622
Epoch 2/5
1500/1500 [==============================] - 1s 603us/step - loss: 0.4019 - accuracy: 0.8539
Epoch 3/5
1500/1500 [==============================] - 1s 598us/step - loss: 0.3532 - accuracy: 0.8719
Epoch 4/5
1500/1500 [==============================] - 1s 590us/step - loss: 0.3298 - accuracy: 0.8808
Epoch 5/5
1500/1500 [==============================] - 1s 584us/step - loss: 0.3233 - accuracy: 0.8838


# 옵티마이저
다양한 종류의 경사 하강법 알고리즘  
`compile()` 메서드에서는 케라스 기본 경사 하강법 알고리즘인 RMSprop을 사용한다 (RMSprop 학습률 또한 조정할 하이퍼파라미터).   

## Adagrad 
learning_rate : 학습률 지정 ( 기본 0.001 )  
Adagrad는 그레디언트 제곱을 누적해 학습률을 나눈다.   
initial_accumulator_value 매개변수에서 누적 초깃값을 지정할 수 있다 ( 0.1 )  

## RMSprop
learning_rate : 학습률 지정 ( 기본 0.001 )  
Adagrad처럼 그레디언트 제곱으로 학습률을 나누지만, 최신 그레디언트를 사용하기 위해 지수 감소를 사용한다.  
rho 매개변수에서 감소 비율을 지정한다. ( 0.9 )   

## Adam
learning_rate : 학습률 지정 ( 기본 0.001 )   
모멘텀 최적화에 있는 그레디언트 지수 감소 평균을 조절하기 위해 beta_1 매개변수가 있다. ( 0.9 )  
RMSprop에 있는 그레디언트 제곱의 지수 감소 평균을 조절하기 위해 beta_2 매개변수가 있다. ( 0.999 )  

In [16]:
# 기본 학습률 값 0.01 
# momentum 0보다 큰 값을 지정해, 이전 그레디언트를 가속도처럼 사용 ( 보통 0.9 이상 지정 )
# nesterov 네스테로프 모멘텀 최적화

sgd = keras.optimizers.SGD(learning_rate= 0.1, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss='sparse_categorical_crossentropy', metrics = 'accuracy')

####  적응적 학습률 Adaptive Learning Rate
모델이 최적점에 가까이 갈수록 학습률을 낮추면 안정적으로 최적점에 수렴할 가능성이 높다  
대표적인 옵티마이저
1. adagrad
2. rmsprop

In [17]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics = 'accuracy')

In [18]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics = 'accuracy')

### Adam
모멘컴 최적화와 RMSprop의 장접을 접목시킨 옵티마이저.

In [19]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5) 

Epoch 1/5
1500/1500 [==============================] - 1s 459us/step - loss: 0.6858 - accuracy: 0.7652
Epoch 2/5
1500/1500 [==============================] - 1s 457us/step - loss: 0.4023 - accuracy: 0.8582
Epoch 3/5
1500/1500 [==============================] - 1s 457us/step - loss: 0.3573 - accuracy: 0.8730
Epoch 4/5
1500/1500 [==============================] - 1s 459us/step - loss: 0.3222 - accuracy: 0.8838
Epoch 5/5
1500/1500 [==============================] - 1s 458us/step - loss: 0.3035 - accuracy: 0.8905


In [21]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 368us/step - loss: 0.3363 - accuracy: 0.8781


[0.336306631565094, 0.878083348274231]